# Data Transformation

We need our data to look something like this for py-irt

```json
{"subject_id": "pedro",    "responses": {"q1": 1, "q2": 0, "q3": 1, "q4": 0}}
{"subject_id": "pinguino", "responses": {"q1": 1, "q2": 1, "q3": 0, "q4": 0}}
{"subject_id": "ken",      "responses": {"q1": 1, "q2": 1, "q3": 1, "q4": 1}}
{"subject_id": "burt",     "responses": {"q1": 0, "q2": 0, "q3": 0, "q4": 0}}
```


## Setup

```
conda create -n ENVNAME python=3.10 pandas
conda activate ENVNAME
```

```
conda install -c conda-forge jupyterlab
pip install pandas pyro-ppl py-irt
```

In [14]:
!pip install pandas pyro-ppl py-irt


Defaulting to user installation because normal site-packages is not writeable


In [15]:
import pandas as pd
import json

In [16]:
D = pd.read_csv("cola.rp", sep="\t", names=["modelID","itemID","response"])

D.head()

,modelID,itemID,response
0,m-0.34,0,1
1,m-0.34,1,1
2,m-0.34,2,1
3,m-0.34,3,1
4,m-0.34,4,1


In [17]:
D.head(-10)

,modelID,itemID,response
0,m-0.34,0,1
1,m-0.34,1,1
2,m-0.34,2,1
3,m-0.34,3,1
4,m-0.34,4,1
...,...,...,...
427535,m-0.04,8536,1
427536,m-0.04,8537,1
427537,m-0.04,8538,1
427538,m-0.04,8539,0


In [18]:
len(D['modelID'].unique())

50

In [19]:
response_patterns = {}

for idx, row in D.iterrows():
    if row["modelID"] not in response_patterns:
        response_patterns[row["modelID"]] = {}
    response_patterns[row["modelID"]][f"q{row['itemID']}"] = row["response"]

In [20]:
with open("cola_pyirt.jsonlines", "w") as outfile:
    for key, val in response_patterns.items():
        outrow = {"subject_id": key, "responses": val}
        outfile.write(json.dumps(outrow) + "\n")

In [21]:
!py-irt train 1pl cola_pyirt.jsonlines output/cola1pl

[08:09:50] config: model_type='1pl' epochs=2000 priors=None           ]8;id=404654;file:///home/lalor/.local/lib/python3.10/site-packages/py_irt/cli.py\cli.py]8;;\:]8;id=230324;file:///home/lalor/.local/lib/python3.10/site-packages/py_irt/cli.py#109\109]8;;\
           initializers=[] dims=None lr=0.1 lr_decay=0.9999                     
           dropout=0.5 hidden=100 vocab_size=None log_every=100                 
           seed=None deterministic=False                                        
           data_path: cola_pyirt.jsonlines                            ]8;id=468206;file:///home/lalor/.local/lib/python3.10/site-packages/py_irt/cli.py\cli.py]8;;\:]8;id=126688;file:///home/lalor/.local/lib/python3.10/site-packages/py_irt/cli.py#111\111]8;;\
           output directory: output/cola1pl                           ]8;id=97943;file:///home/lalor/.local/lib/python3.10/site-packages/py_irt/cli.py\cli.py]8;;\:]8;id=553054;file:///home/lalor/.local/lib/python3.10/s

For the next block, use either

```
!cat output/cola1pl/best_parameters.json
```

or

```
!type output\cola1pl\best_parameters.json
```

depending on your operating system.

In [32]:
with open ("output/cola1pl/best_parameters.json","r") as file:
    data = json.load(file)
data.keys()

dict_keys(['ability', 'diff', 'irt_model', 'item_ids', 'subject_ids'])

In [33]:
len(data["diff"])

8551

In [39]:
items = pd.DataFrame(
    index=data["item_ids"], 
    data= data["diff"],
    columns=["diff"]
)
items.head()

,diff
0,-16.024603
1,-14.949307
2,-20.603096
3,-22.106720
4,-17.044914


In [41]:
subjects = pd.DataFrame(
    index=data["subject_ids"], 
    data= data["ability"],
    columns=["ability"]
)
subjects.head()

,ability
0,9.435297
1,10.093401
2,8.901505
3,9.171259
4,8.865317
